# CVL Visualization Examples

This notebook illustrates how to visualize GeoJSON, raster images and 3D geometry using the CVL 3D globe. Open https://cvl.eo.eas.int/ or https://nlive.norceresearch.no/cvl/ in a separate browser tab, then step through the notebook.

## Setup

We start by importing some modules and defining some helper functions.

In [ ]:
from viz import viz, VBO, Raster
import os
import sys
import traceback
import numpy as np
import math
import time
import io
from osgeo import osr
from osgeo import gdal

visualizer = viz()


### post_3d_data

This function is used to post 3D data to the visualization. The parameters are:

- `key` : A key that is used to identify the object on the server.
- `primitive` : The 3D primitive we want to render. Can be: `points`, `lines`, `linestrip`, `lineloop`, `triangles`, `trianglestrip`
- `vertices` : (N,3), dtype=float64 array holding projected vertex data
- `color` : (N, 1), dtype=uint32 array holding color values
- `texcoord` : (N, 2), dtype=float32 array holding texture coordinates
- `normal` : (N, 3), dtype=float32 array containing normalized vertex normals
- `index` : (?,), dtype=uint32 array containing indices. If no index buffer is specified, an implicit index buffer containing the indices from 0 to N-1 is used.
- `texture` : A bytes object containing texture data. The format of the texture must be either PNG or JPEG.
- `compute_normals` : Only applicable when rendering triangles. Attempts to compute vertex normals based on the provided vertices and indices. Currently a bit unstable.

In [ ]:
def post_3d_data(key, primitive, vertices, color=None, texcoord=None, normal=None, index=None, texture=None, compute_normals=False):
    #metadata = { "path" : "Examples", "time_start" : time.time()-601, "time_stop" : time.time()-601 }
    metadata = { "path" : "Examples" }
    vbo = VBO(primitive, projection=32633)
    vbo.compute_normals = compute_normals
    vbo.set_vertex(vertices)
    vbo.set_color(color)
    vbo.set_texcoord(texcoord)
    vbo.set_normal(normal)
    vbo.set_index(index)
    vbo.texture = texture
    visualizer.publish_vbo(key, metadata, vbo)

### gen_index

This function generates an index buffer rendering triangles. It assumes that the vertices are laid out as in a regular grid. Input parameters are the width and height of the grid.

In [ ]:
def gen_index(width, height):
    verts_per_line = 2*width
    tris_per_line = verts_per_line-2
    num_tris = tris_per_line*(height-1)
    num_index = num_tris*3
    indices = np.zeros((num_index), dtype=np.uint32)
    idx = 0
    for y in range(0, height-1):
        for x in range(0, width-1):
            indices[idx+0]	= ((y+1) * width) + x
            indices[idx+1]	= (y*width)+x
            indices[idx+2]	= (y*width)+x+1
            indices[idx+3]	= (y*width)+x+1
            indices[idx+4]	= ((y+1) * width) + x+1
            indices[idx+5]	= ((y+1) * width) + x
            idx += 6
    return indices


## Rendering points and triangles

Here we generate a bunch of points centered around a geographical origin. We also generate a color for each vertex.

In [ ]:
origin = np.array((509389, 8686070, 700)) # The origin is in UTM 33.

w = 100
h = 100
z = 0
scale = 100
num_points = w*h
points = np.zeros((num_points, 3), dtype=np.float64)
colors = np.zeros((w*h,), dtype=np.uint32)
blue = 255
alpha = 255
for y in range(0, h):
    yf = y/h
    green = int(yf*255.0)
    for x in range(0, w):
        xf = x/w
        xg = (xf-0.5)*2
        yg = (yf-0.5)*2
        d = math.sqrt(xg**2+yg**2)
        index = (x+y*w)
        points[index] = [ x*scale, y*scale, math.exp(-(d**2)/0.25)*5000]
        red = int(xf*255.0)
        # We use the VBO.rgb_to_color function to convert values in
        # the range 0-255 to an appropriate uint32 color value. It is
        # also possible to specify an alpha value (also in the range 0-255).
        colors[index] = VBO.rgb_to_color(red, green, blue, alpha)

# Finally, post the data we just created, first as points...        
post_3d_data("Example points", "points", points+origin, colors)
# Then generate an index buffer and render the same geometry, slightly offset, as triangles
index = gen_index(w, h)
post_3d_data("Example mesh", "triangles", points+origin+[w*1.1*scale, 0, 600], colors, index=index, compute_normals=True)


### Drawing lines

In [ ]:
points = np.zeros((4, 3), dtype=np.float64)
colors = np.zeros((4), dtype=np.uint32)
points[0] = origin
points[1] = origin+[1000, 0, 0]
points[2] = origin+[1000, 1000, 0]
points[3] = origin+[0, 1000, 0]
colors[0] = 0xffffffff
colors[1] = 0xffffffff
colors[2] = 0xffffffff
colors[3] = 0xffffffff

post_3d_data("Example lines", "lineloop", points, colors)


### Adding a texture

In [ ]:
# Note that the ordering of the points is different here, compared to the lines above.
points[0] = origin
points[1] = origin+[1000, 0, 0]
points[2] = origin+[0, 1000, 0]
points[3] = origin+[1000, 1000, 0]

texcoords = np.zeros((4, 2), dtype=np.float32)
texcoords[0] = [0, 0]
texcoords[1] = [1, 0]
texcoords[2] = [0, 1]
texcoords[3] = [1, 1]
index = gen_index(2,2)

with open("texture.jpg", "rb") as fd:
    texture = fd.read()

post_3d_data("Example texture", "triangles", points+[1200,0,0], None, texcoord=texcoords, index=index, texture=texture)

## GeoJSON

Visualizing GeoJSON is easy. First, some setup to reproject from UTM 33 to WGS 84:

In [ ]:
must_set_axis_mapping = int(gdal.VersionInfo()) > 3 * 1000000
src = osr.SpatialReference()
src.ImportFromEPSG(32633)
tgt = osr.SpatialReference()
tgt.ImportFromEPSG(4326)
if must_set_axis_mapping:
    tgt.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)
xform = osr.CoordinateTransformation(src, tgt)

Next, we create function that returns a polygon according to the GeoJSON specification. As part of this, generated points are reprojected to WGS84:

In [ ]:
def create_polygon(origin, size):
    polygon = [ (origin[0]-size, origin[1]-size),
                (origin[0]+size, origin[1]-size),
                (origin[0]+size, origin[1]+size),
                (origin[0]-size, origin[1]+size)]
    polygon = xform.TransformPoints(polygon)
    # Repeat the initial point
    polygon.append(polygon[0])
    return [ polygon ]

Finally, create a dictionary holding the GeoJSON itself, and embed it in the metadata for the object. Note that we can specify the stroke color, stroke width and fill color by attaching the properties `stroke`, `fill` and `strokeWidth` to the `properties` part of any given GeoJSON `Feature`. The allowed values for `stroke` and `fill` are valid CSS colors (e.g `#rrggbb`, `#rgb`, `rgba(r, g, b, a)`). For the latter syntax, red, green and blue values range from 0-255 and the alpha value ranges from 0-1.

In [ ]:
geojson = { "type" : "Feature",
            "geometry" : {
                "type" : "Polygon",
                "coordinates" : create_polygon(origin, 2500) },
             "properties" : { "info" : "A particularly interesting area",
                              "stroke" : "#fff",
                              "fill" : "rgba(128, 0, 0, 0.5)",
                              "strokeWidth" : "5.0"} }
metadata = { "path" : "Examples", "geojson" : geojson }
print(metadata)
visualizer.publish_geojson("Example GeoJSON", metadata)

## Raster

Publishing a raster image requires building a reprojection grid that will be used when visualizing the image. The grid can be specified to an arbitrary resolution. Bi-linear interpolation on the GPU is used to stretch the image's pixels between 4 neighbouring grid points.

In [ ]:
# We still have our points in the order we want them,
# but we have no need for the Z (height) coordinate any longer
points = np.zeros((9,3), dtype=np.float64)
for y in range(0,3):
    for x in range(0,3):
        points[y*3+x] = origin+[500*x, 500*y, 0]

raster = Raster(np.array(points[:, 0:2]), [3,3], 32633, image_data=texture)
metadata = { "path" : "Examples" }
visualizer.publish_raster("A raster image", metadata, raster)

# Timeseries

Any of the objects above can be incorporated in a timeseries by including the fields `time_start` and `time_stop` in the metadata associated with an object. The timestamp must be an epoch timestamp in the UTC time zone.

The following example illustrates a GeoJSON timeseries using the polygon function we created earlier. By adjusting the time window exposed by the visualization or moving the time slider, different parts of the data can be visualized.


In [ ]:
timestamp = time.time()
for i in range(0, 100):
    origin = np.array((509389-2000, 8686070, 0))
    factor = i/50.-1.
    metadata = { "path" : "Examples", "name" : "Timeseries GeoJSON" }
    geojson = { "type" : "Feature",
            "geometry" : {
                "type" : "Polygon",
                "coordinates" : create_polygon(origin+(factor*5000, factor*5000, 0), 25+abs(factor*25)) },
             "properties" : { "stroke" : "#fff",
                              "fill" : None,
                              "strokeWidth" : "1.0"} }
    metadata["geojson"] = geojson
    metadata["time_start"] = timestamp-i*10
    metadata["time_stop"] = timestamp-i*10
    visualizer.publish_geojson(f"timeseries-geojson-{i}", metadata)


# Control

The visualization can be controlled from Python by calling various methods on the visualizer object. Any updates made in this manner occur in realtime. The available methods are:

- `look_at(eye, target, duration)`
- `set_time(timestamp)`
- `set_time_window(window_size)`
- `query()`

The `query()` method returns an array containing the current view and timing state of all currently connected visualization frontends.


In [ ]:
visualizer.look_at([19.115, 69.50937, 5000], [18.953, 69.67313, 0], 1)
now = time.time()
one_day = 24*60*60
visualizer.set_time(now-60*60*4)
visualizer.set_time_window(600)
result = visualizer.query()
print(result)

# Metadata Reference

Some fields in an object's metadata have special meanings. The special fields are described below.

## `name`

If this field is present, this object will be grouped with others sharing the same `name` in the user interface. It is a good idea for timeseries or other datasets where you expect to have many objects (more 20-50) to use group objects in using this field, since the user interface can become a bottleneck when updating or rendering. If an object doesn't have a `name`, the object's `key` is used to name it in the user interface instead. (Note that the `key` isn't part of the metadata - it is specified separately when publishing an object.)

## `time_start` and `time_end`

To have timeseries appear in the timeline and be subject to the clock (automatically hide elements outside the current time window), these fields must be present.

## `path`

You can use the `path` field to structure the user interface hierarchically. An empty or missing path will place objects at the top level of the user interface. To add directories, simply specify a path like `Examples.Raster`, which will create a tree two levels deep containing the object(s) with the same `path` within.

# Threading Notes

The viz object defaults to using threads to process and post updates to the server. Your code should be aware of this fact, and treat items that are posted as immutable. If you can't do this, initialize the visualizer instance with viz(threaded=False).